In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
from descriptors import sift_descriptor, brisk_descriptor
from hilbert_curve import *
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image

In [2]:
sift_label = []
sift_data = []

hilbert_label = []
hilbert_data = []

#p_class1 = Path('../dtd/images/bubbly')
#p_class2 = Path('../dtd/images/cracked')

p_class1 = Path('../img/clean-bg/motorbike')
p_class2 = Path('../img/clean-bg/airplane')

# n_images = min(len(list(p_bubbly.glob('*'))), len(list(p_cracked.glob('*'))))
n_images = 100

### Descritores e labels da classe 1

In [3]:
for f in list(p_class1.iterdir())[:n_images]:
    
    if f.suffix in ['.jpg', '.png', '.jpeg']:
    
        img = cv.imread(str(f))    

        sift_kp, sift_desc = sift_descriptor(img)
        
        hilbert_desc = image_descriptor(Image.fromarray(img).convert('L'), keypoints=sift_kp)

        sift_data.extend(sift_desc)
        hilbert_data.extend(hilbert_desc)

        sift_label.extend([0]*len(sift_desc)) # 0 => motorbike
        hilbert_label.extend([0]*len(hilbert_desc))


### Descritores e labels da classe 2

In [4]:
for f in list(p_class2.iterdir())[:n_images]:
    
    if f.suffix in ['.jpg', '.png', '.jpeg']:
    
        img = cv.imread(str(f)) 

        sift_kp, sift_desc = sift_descriptor(img)
        
        hilbert_desc = image_descriptor(Image.fromarray(img).convert('L'), keypoints=sift_kp)

        sift_data.extend(sift_desc)
        hilbert_data.extend(hilbert_desc)

        sift_label.extend([1]*len(sift_desc)) # 1 => airplane
        hilbert_label.extend([1]*len(hilbert_desc))
    

In [5]:
sift_desc.shape

(277, 128)

In [6]:
hilbert_desc.shape

(232, 6)

In [7]:
sift_label = np.asarray(sift_label)
hilbert_label = np.asarray(hilbert_label)

### K-Nearest Neighbors usando descritor SIFT

In [8]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from joblib import dump, load

In [9]:
X_sift_train, X_sift_test, y_sift_train, y_sift_test = train_test_split(sift_data, sift_label)

knn_sift = KNeighborsClassifier(n_neighbors=3)

knn_sift.fit(X_sift_train, y_sift_train)
predict_sift = knn_sift.predict(X_sift_test)

report_sift = classification_report(y_sift_test, predict_sift, output_dict=True, target_names=['motorbike', 'airplane'])
print(classification_report(y_sift_test, predict_sift, target_names=['motorbike', 'airplane']))

              precision    recall  f1-score   support

   motorbike       0.80      0.76      0.78      4859
    airplane       0.69      0.73      0.71      3546

    accuracy                           0.75      8405
   macro avg       0.74      0.75      0.75      8405
weighted avg       0.75      0.75      0.75      8405



### K-Nearest Neighbors usando descritor de curvas de Hilbert

In [10]:
X_hilbert_train, X_hilbert_test, y_hilbert_train, y_hilbert_test = train_test_split(np.asarray(hilbert_data), hilbert_label)

knn_hilbert = KNeighborsClassifier(n_neighbors=3)
knn_hilbert.fit(X_hilbert_train, y_hilbert_train)

predict_hilbert = knn_hilbert.predict(X_hilbert_test)

report_hilbert = classification_report(y_hilbert_test, predict_hilbert, output_dict=True, target_names=['motorbike', 'airplane'])
print(classification_report(y_hilbert_test, predict_hilbert, target_names=['motorbike', 'airplane']))

              precision    recall  f1-score   support

   motorbike       0.88      0.85      0.87      4853
    airplane       0.79      0.83      0.81      3243

    accuracy                           0.84      8096
   macro avg       0.83      0.84      0.84      8096
weighted avg       0.84      0.84      0.84      8096



Saving models

In [19]:
dump(knn_sift, '../models/knn_sift.z', compress=3)
dump(knn_hilbert, '../models/knn_hilbert.z', compress=3)

['../models/knn_hilbert.z']

### Gerar tabela LaTeX com métricas dos modelos 

In [11]:
import pandas as pd

In [42]:
report_all = {'SIFT':{}, 'Hilbert':{}}

report_all['SIFT']['accuracy'] = np.round(report_sift['accuracy'], 2)
report_all['Hilbert']['accuracy'] = np.round(report_hilbert['accuracy'], 2)

for key in ['precision', 'recall', 'f1-score']:
    report_all['SIFT'][key] = np.round(report_sift['weighted avg'][key], 2)
    report_all['Hilbert'][key] = np.round(report_hilbert['weighted avg'][key], 2)
    

In [44]:
df_report_all = pd.DataFrame(report_all).transpose()

In [45]:
latex_report = df_report_all.to_latex(caption = "Comparação das métricas dos modelos KNN utilizando descritores SIFT e de Hilbert")

In [46]:
print(latex_report)

\begin{table}
\centering
\caption{Comparação das métricas dos modelos KNN utilizando descritores SIFT e de Hilbert}
\begin{tabular}{lrrrr}
\toprule
{} &  accuracy &  precision &  recall &  f1-score \\
\midrule
SIFT    &      0.75 &       0.75 &    0.75 &      0.75 \\
Hilbert &      0.84 &       0.84 &    0.84 &      0.84 \\
\bottomrule
\end{tabular}
\end{table}

